In [ ]:
###Fake news detection


In [34]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [ ]:
fake_news=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fake.csv')
true_news=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/True.csv')

In [ ]:
fake_news.shape,true_news.shape


((23481, 4), (21417, 4))

In [ ]:
fake_news["label"]=0
true_news["label"]=1

In [ ]:
fake_manual_testing=fake_news.tail(1000)
for i in range(23480,22480,-1):
    fake_news.drop([i],axis=0,inplace=True)
true_manual_testing=true_news.tail(1000)
for i in range(21416,20416,-1):
    true_news.drop([i],axis=0,inplace=True)

In [ ]:
manual_testing=pd.concat([true_manual_testing, fake_manual_testing], axis=0)

In [ ]:
manual_testing.to_csv("Manual_Testing.csv")

In [ ]:
marged_data=pd.concat([fake_news, true_news],axis=0)
marged_data.to_csv("Marged_Data.csv")

In [ ]:
temp=marged_data.drop(["title","subject","date"],axis=1)

In [ ]:
temp=temp.sample(frac=1)

In [ ]:
temp.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
###remove unnecessary content like extra space etc.

def word_process(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text= re.sub("\\W"," ",text)
    text=re.sub('http?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]' %re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    return text

In [ ]:
temp["text"]=temp["text"].apply(word_process)

In [ ]:
x=temp["text"]
y=temp["label"]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [ ]:
###vectorize
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorization =TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

In [ ]:
###logistic regression
from sklearn.linear_model import LogisticRegression

In [ ]:
LR=LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
LogisticRegression()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
LR.score(xv_test,y_test)

0.9868531468531468

In [ ]:
predict_LR=LR.predict(xv_test)
prob_LR=LR.predict_proba(xv_test)

In [ ]:
print(classification_report(y_test,predict_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5529
           1       0.99      0.98      0.99      5196

    accuracy                           0.99     10725
   macro avg       0.99      0.99      0.99     10725
weighted avg       0.99      0.99      0.99     10725



Random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC=RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
RFC.score(xv_train,y_train)

1.0

In [ ]:
predict_rfc=RFC.predict(xv_test)
prob_rfc=RFC.predict_proba(xv_test)

In [ ]:
print(classification_report(y_test,predict_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5529
           1       0.99      0.98      0.99      5196

    accuracy                           0.99     10725
   macro avg       0.99      0.99      0.99     10725
weighted avg       0.99      0.99      0.99     10725



Manual Testing


In [ ]:
###manual testing

def output_label(n):
    if n==0:
        return "Fake news"
    else:
        return "True"
def manual_testing(news):
    testing_news={"text":[news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test["text"]=new_def_test["text"].apply(word_process)
    new_x_test=new_def_test["text"]
    new_xv_test= vectorization.transform(new_x_test)
    predict_LR= LR.predict(new_xv_test)
    predict_rfc=RFC.predict(new_xv_test)
    prob_rfc=RFC.predict_proba(new_xv_test)
    prob_LR=LR.predict_proba(new_xv_test)
    a=prob_LR[0][1]
    a*=100
    b=prob_rfc[0][1]
    b*=100
    
    print("\nLR Prediction: {}".format(output_label(predict_LR))) 
    print("\nLR Probability of Truthness %0.2f"%a +" %")  
    print("\nRFC Probability of Truthness %0.2f"%b +" %")
    return print("\nRFC Prediction: {}".format(output_label(predict_rfc)))

In [ ]:
input_news=input(str())
manual_testing(input_news)

LIMA (Reuters) - Six farmers were shot dead in the Peruvian Amazon by a group of masked men in an apparent dispute over land rights, a police officer and a tribal leader said on Thursday. Five bodies, one with hands and feet bound, had been thrown in a river and a sixth was found by the side of an unpaved road in the jungle region Ucayali, police officer Raul Huari said. The victims were part of a community of peasant farmers that had refused to leave the lands they work on when pressured by oil palm growers, said Robert Guimaraes, the head of an indigenous federation in Ucayali. Witnesses testified that a group of between 30 and 40 men carrying shotguns tried to kill some 20 farmers altogether, Huari said.  They said they showed up, surrounded them and just started shooting. Fortunately, some managed to escape,  he said.  In my 24 years of working I ve never seen anything like this.  Huari said the murders appeared to be linked to a land dispute, and a special police unit and prosecut